In [19]:
##################
# Load libraries #
##################

from get_strava_data import my_data, process_data, get_elevation

import requests

import pandas as pd
import numpy as np

from PIL import Image
import base64

import ast
import polyline

import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import IFrame
from folium.features import CustomIcon


In [20]:
print("folium.__version__",folium.__version__)

folium.__version__ 0.12.1.post1


In [21]:
polylines_df = pd.read_csv('./data/processed_data.csv', usecols=['name', 'distance', 'total_elevation_gain', 'average_speed', 'weighted_average_watts', 'year', 'month', 'day', 'type', 'map'])
polylines_df = polylines_df[polylines_df.type == 'Ride'] # We'll only use rides which have a map

ride_names = polylines_df.name


# We'll choose rides by index to avoid duplicate names 
ride_index = 3
decoded = pd.json_normalize(polylines_df[polylines_df.index == ride_index]['map'].apply(ast.literal_eval))['summary_polyline'].apply(polyline.decode).values[0]


In [22]:
decoded[0]

(37.78761, -122.27535)

In [23]:
polylines_df[polylines_df.name == 'Monterey - Butters']

,name,distance,total_elevation_gain,type,map,average_speed,weighted_average_watts,year,month,day
3,Monterey - Butters,20.4,1355.0,Ride,"{'id': 'a6955866591', 'summary_polyline': 'qks...",13.9,166.0,2022,4,9


In [24]:
polylines_df[polylines_df.name == 'Monterey - Butters'].index.values[0]

3

In [25]:
# Mapping route with folium

image_file = './data/elevation_profile.png'
encoded = base64.b64encode(open(image_file, 'rb').read()).decode('UTF-8')

html = '<img src="data:image/png;base64,{}">'.format
iframe = IFrame(html(encoded), width=632+20, height=420+20)
popup = folium.Popup(iframe, max_width=2650)

centroid = [
    np.mean([coord[0] for coord in decoded]), 
    np.mean([coord[1] for coord in decoded])
]
my_map = folium.Map(location=centroid, zoom_start=13, popup=popup)
folium.PolyLine(decoded).add_to(my_map)

my_map

TypeError: Object of type Popup is not JSON serializable

In [8]:
# Reading data from csv
# For maps we're interested in the map column with the polyline encoded

# We'll use the polyline library to decode the polyline
polylines_df = pd.read_csv('./data/processed_data.csv', usecols=['name', 'distance', 'total_elevation_gain', 'weighted_average_watts', 'year', 'month', 'day', 'type', 'map'])
polylines_df = polylines_df[polylines_df.type == 'Ride'] # We'll only use rides which have a map

polylines_df['map'] = polylines_df['map'].apply(ast.literal_eval) # The maps column is a string, so we need to convert it to a dictionary
polylines_df[['id', 'summary_polyline', 'a']] = pd.json_normalize(polylines_df['map']) # Splitting the dictionary into separate columns

polylines_df.drop(columns=['type', 'map', 'a'], inplace=True) 
polylines_df = polylines_df[polylines_df.summary_polyline.notna()] # Dropping rows with no polyline
polylines_df['date'] = pd.to_datetime(polylines_df[['year', 'month', 'day']])

# Decoding the polyline
polylines_df['decoded_polyline'] = polylines_df['summary_polyline'].apply(polyline.decode)
# polylines_df.drop(columns=['summary_polyline'], inplace=True)
# Saving to CSV
polylines_df.to_csv('./data/decoded_polylines.csv')
polylines_df.head()




,name,distance,total_elevation_gain,weighted_average_watts,year,month,day,id,summary_polyline,date,decoded_polyline
1,Afternoon Ride,18.1,167.3,153.0,2022,4,10,a6956840241,museFnqjiVAjDMnAGx@?d@CvC@NDPEnA@TFd@LZb@`@dEt...,2022-04-10,"[(37.78919, -122.28392), (37.7892, -122.28478)..."
2,Fun with the kiddos,4.0,7.5,NaN,2022,4,9,a6955866591,qkseF|{hiVj@o@~BqAl@q@^_AjBiH^{@^e@nDuC`Ag@nCW...,2022-04-09,"[(37.78761, -122.27535), (37.78739, -122.27511..."
3,Monterey - Butters,20.4,1355.0,166.0,2022,4,9,a6947233958,okseF|{hiV`AaArBkAj@s@j@_B|AcGXm@`@g@tAoAfCmBx...,2022-04-09,"[(37.7876, -122.27535), (37.78727, -122.27502)..."
5,Hot! Too hot! 🥵,18.3,128.0,130.0,2022,4,7,a6933025280,ckseF|{hiV`@c@d@a@PMr@[\UZ]Ta@Vm@|@{DXcAVu@\q@...,2022-04-07,"[(37.78754, -122.27535), (37.78737, -122.27517..."
7,Windy! 💨,19.3,265.7,152.0,2022,4,5,a6922321736,ikseF~{hiVxEaEzDkLzEaErFs@|AaFxCgD`AeG@kM~DKrA...,2022-04-05,"[(37.78757, -122.27536), (37.78648, -122.27439..."


In [25]:
# # Reading data from csv
# # For maps we're interested in the map column with the polyline encoded

# # We'll use the polyline library to decode the polyline
# map_df = pd.read_csv('./data/processed_data.csv', usecols=['name', 'distance', 'total_elevation_gain', 'weighted_average_watts', 'year', 'month', 'day', 'type', 'map'])
# map_df = map_df[map_df.type == 'Ride'] # We'll only use rides which have a map


# map_df['map'] = map_df['map'].apply(ast.literal_eval) # The maps column is a string, so we need to convert it to a dictionary

# map_df[['id', 'summary_polyline', 'a']] = pd.json_normalize(map_df['map']) # Splitting the dictionary into separate columns

# map_df.drop(columns=['type', 'map', 'a'], inplace=True) 
# polylines_df = map_df.dropna() # Dropping rows with no polyline
# polylines_df['date'] = pd.to_datetime(polylines_df[['year', 'month', 'day']])

# # Decoding the polyline
# polylines_df['decoded_polyline'] = polylines_df['summary_polyline'].apply(polyline.decode)
# polylines_df.drop(columns=['summary_polyline'], inplace=True)
# # Saving to CSV
# polylines_df.to_csv('./data/decoded_polylines.csv')
# polylines_df.head()


In [9]:
# Selecting date
name = 'Monterey - Butters'

polylines_df[polylines_df.name == name]['decoded_polyline'].values[0]

[(37.7876, -122.27535),
 (37.78727, -122.27502),
 (37.78669, -122.27464),
 (37.78647, -122.27438),
 (37.78625, -122.2739),
 (37.78578, -122.2726),
 (37.78565, -122.27237),
 (37.78548, -122.27217),
 (37.78505, -122.27177),
 (37.78437, -122.27122),
 (37.78408, -122.27114),
 (37.7833, -122.27109),
 (37.78317, -122.27103),
 (37.78309, -122.27088),
 (37.78277, -122.26996),
 (37.78262, -122.26974),
 (37.78228, -122.26943),
 (37.78205, -122.26917),
 (37.78191, -122.26893),
 (37.78182, -122.26867),
 (37.78166, -122.26784),
 (37.78159, -122.26716),
 (37.78168, -122.26603),
 (37.78168, -122.26549),
 (37.78164, -122.26543),
 (37.78153, -122.26539),
 (37.78091, -122.26541),
 (37.78069, -122.26537),
 (37.78062, -122.26527),
 (37.78045, -122.26464),
 (37.7803, -122.26433),
 (37.77995, -122.26387),
 (37.77969, -122.26362),
 (37.77963, -122.26363),
 (37.77962, -122.26402),
 (37.77972, -122.26611),
 (37.77972, -122.26751),
 (37.77983, -122.269),
 (37.77982, -122.27005),
 (37.77991, -122.27105),
 (37.77

In [27]:
#################################
# TESTING MAPPING FUNCTIONALITY #
#################################

# Selecting ride by name
name = 'Monterey - Butters'

# Testing my mapping functionality
decoded = polylines_df[polylines_df.name == name]['decoded_polyline'].values[0]

# Adding elevation data from Open Street Map
# elevation_profile = [get_elevation(coord[0], coord[1]) for coord in decoded]
# elevation_profile_feet = [elevation_profile[i] * 3.28084 for i in range(len(elevation_profile))] # Converting elevation to feet

In [18]:
# elevation_profile_feet

In [19]:
# sns.lineplot(x=range(len(elevation_profile_feet)), y=pd.Series(elevation_profile_feet).rolling(5).mean())

In [20]:
# # Plotting elevation data
# fig, ax = plt.subplots(figsize=(10, 4))
# ax = pd.Series(elevation_profile_feet).rolling(3).mean().plot(
#     ax=ax, 
#     color='steelblue', 
#     legend=False
# )
# ax.set_ylabel('Elevation (ft)')
# ax.axes.xaxis.set_visible(False)
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)
# plt.show()

In [10]:
# Mapping route with folium

centroid = [
    np.mean([coord[0] for coord in decoded]), 
    np.mean([coord[1] for coord in decoded])
]
my_map = folium.Map(location=centroid, zoom_start=13)
folium.PolyLine(decoded).add_to(my_map)


icon = './icons/bike_ride.png'
icon_image = Image.open(icon)
        
icon = CustomIcon(
np.array(icon_image),
icon_size=(50, 50),
popup_anchor=(0, -30),
)


html = '''1st line<br>
2nd line<br>
3rd line'''

iframe = folium.IFrame(html,
                       width=100,
                       height=100)

popup = folium.Popup(iframe,
                     max_width=100)

marker = folium.Marker(polylines_df[polylines_df.name == name]['decoded_polyline'].values[0][0],
                       popup=popup, icon=icon).add_to(my_map)





my_map

In [12]:
# Decoding the polyline
polylines_df['decoded_polyline'] = polylines_df['summary_polyline'].apply(polyline.decode)
polylines_df.head()

,name,distance,total_elevation_gain,weighted_average_watts,year,month,day,id,summary_polyline,date,decoded_polyline
1,Afternoon Ride,18.1,167.3,153.0,2022,4,10,a6956840241,museFnqjiVAjDMnAGx@?d@CvC@NDPEnA@TFd@LZb@`@dEt...,2022-04-10,"[(37.78919, -122.28392), (37.7892, -122.28478)..."
2,Fun with the kiddos,4.0,7.5,NaN,2022,4,9,a6955866591,qkseF|{hiVj@o@~BqAl@q@^_AjBiH^{@^e@nDuC`Ag@nCW...,2022-04-09,"[(37.78761, -122.27535), (37.78739, -122.27511..."
3,Monterey - Butters,20.4,1355.0,166.0,2022,4,9,a6947233958,okseF|{hiV`AaArBkAj@s@j@_B|AcGXm@`@g@tAoAfCmBx...,2022-04-09,"[(37.7876, -122.27535), (37.78727, -122.27502)..."
5,Hot! Too hot! 🥵,18.3,128.0,130.0,2022,4,7,a6933025280,ckseF|{hiV`@c@d@a@PMr@[\UZ]Ta@Vm@|@{DXcAVu@\q@...,2022-04-07,"[(37.78754, -122.27535), (37.78737, -122.27517..."
7,Windy! 💨,19.3,265.7,152.0,2022,4,5,a6922321736,ikseF~{hiVxEaEzDkLzEaErFs@|AaFxCgD`AeG@kM~DKrA...,2022-04-05,"[(37.78757, -122.27536), (37.78648, -122.27439..."


In [13]:
# Testing my mapping functionality
decoded_150 = polylines_df.loc[polylines_df.index == 150]['decoded_polyline']
decoded_150

150    [(40.84903, -73.94115), (40.84907, -73.94165),...
Name: decoded_polyline, dtype: object

In [14]:
# Adding elevation data from Open Street Map
# polylines_df['elevation'] = polylines_df['decoded_polyline'].apply(lambda x: [get_elevation(lat, lon) for lat, lon in x])

In [15]:
# Saving to CSV
polylines_df.to_csv('./data/polylines.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax = pd.Series(elevation).rolling(3).mean().plot(
    ax=ax, 
    color='steelblue', 
    legend=False
)
ax.set_ylabel('Elevation')
ax.axes.xaxis.set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.show()

In [ ]:
centroid = [
    np.mean([coord[0] for coord in polylines_df.decoded_polyline[56]]), 
    np.mean([coord[1] for coord in polylines_df.decoded_polyline[56]])
]
my_map = folium.Map(location=centroid, zoom_start=13)
folium.PolyLine(polylines_df.decoded_polyline[56]).add_to(my_map)
my_map